In [33]:
from funcs import * # funcs include functions used across scripts
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed

read the .ict data

In [2]:
o = read_all('OPTICAL') # optical data
inl = read_all('InletFlag') # inlet flag
a = read_all('AMS') # ams
mn = read_all('MetNav') # met nav

read the DASH data

In [3]:
retrieval_paths = sorted(glob.glob('../data/v2/*/*/*.csv'))
retrieval_list = []
for i in range(0, len(retrieval_paths)):
    retrieval_list.append(pd.read_csv(retrieval_paths[i], parse_dates=['Start_Date_Time_UTC', 'Stop_Date_Time_UTC']))
r = pd.concat(retrieval_list).reset_index(drop=True)

Separate data by size

In [4]:
r_dps = []
dps = sorted(r['SEMS_Dp'].unique())
for i in range(0,len(dps)):
    r_dps.append(r[(r['RH']>75) & (r['SEMS_Dp']==dps[i])])

Rename variables so they have size in name

In [6]:
for i in range(0, len(dps)):
    d_label = str(int(dps[i]))
    r_dps[i] = r_dps[i].rename(columns={'GF_raw':'GF_raw_'+d_label})

In [13]:
def merge_df(df1, df2, st1 = 'Start_Date_Time_UTC', et1 = 'Stop_Date_Time_UTC', st2 = 'Time_Start', et2 = 'Time_Start', keys = ['GPS_Altitude']):
    out = []
    for i in range(0,len(df1)):
        row = df1.iloc[i]
        t_start, t_stop = row[st1], row[et1]
        out.append(df2[(df2[st2] >= t_start) & (df2[et2] <= t_stop)].mean(numeric_only=True))
    df_out = pd.DataFrame(out)
    df1.loc[:,keys] = df_out.loc[:,keys]
    return df1

In [35]:
def merge_df_p(df1, df2, st1 = 'Start_Date_Time_UTC', et1 = 'Stop_Date_Time_UTC', st2 = 'Time_Start', et2 = 'Time_Start', keys = ['GPS_Altitude']):
    
    def process_row(i):
        row = df1.iloc[i]
        t_start, t_stop = row[st1], row[et1]
        return df2[(df2[st2] >= t_start) & (df2[et2] <= t_stop)].mean(numeric_only=True)

    out = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in tqdm(range(0, len(df1))))

    df_out = pd.DataFrame(out)
    df1.loc[:,keys] = df_out.loc[:,keys]
    return df1

In [36]:
d_test = merge_df_p(r_dps[3], mn)

100%|██████████| 23238/23238 [00:09<00:00, 2508.26it/s]


In [37]:
d_test

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_200,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time,GPS_Altitude
0,2024-05-28 11:58:08,2024-05-28 11:58:16,84.744444,200.0,1.470094,1.497329,NaN,NaN,43088,43096,43092.0,106.555556
1,2024-05-28 11:58:16,2024-05-28 11:58:20,83.100000,200.0,NaN,NaN,NaN,NaN,43096,43100,43098.0,146.000000
2,2024-05-28 11:58:20,2024-05-28 11:58:22,82.933333,200.0,1.436631,1.454321,0.976867,NaN,43100,43102,43101.0,165.000000
3,2024-05-28 11:58:30,2024-05-28 11:58:31,84.200000,200.0,NaN,NaN,NaN,NaN,43110,43111,43110.5,229.500000
4,2024-05-28 11:58:34,2024-05-28 11:58:35,82.400000,200.0,NaN,NaN,NaN,NaN,43114,43115,43114.5,258.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
32870,2024-08-15 18:27:24,2024-08-15 18:27:43,84.705000,200.0,1.577168,1.554715,1.711508,NaN,66444,66463,66453.5,NaN
32871,2024-08-15 18:27:43,2024-08-15 18:28:15,84.390909,200.0,1.548301,1.528602,1.390944,NaN,66463,66495,66479.0,NaN
32872,2024-08-15 18:28:15,2024-08-15 18:29:06,84.521154,200.0,1.577168,1.554715,1.074766,NaN,66495,66546,66520.5,NaN
32873,2024-08-15 18:29:06,2024-08-15 18:30:11,85.327273,200.0,1.548301,1.528602,1.710710,NaN,66546,66611,66578.5,NaN


In [25]:
d_test[np.isfinite(d_test['GPS_Altitude'])]

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_200,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time,GPS_Altitude
0,2024-05-28 11:58:08,2024-05-28 11:58:16,84.744444,200.0,1.470094,1.497329,NaN,NaN,43088,43096,43092.0,106.555556
1,2024-05-28 11:58:16,2024-05-28 11:58:20,83.100000,200.0,NaN,NaN,NaN,NaN,43096,43100,43098.0,146.000000
2,2024-05-28 11:58:20,2024-05-28 11:58:22,82.933333,200.0,1.436631,1.454321,0.976867,NaN,43100,43102,43101.0,165.000000
3,2024-05-28 11:58:30,2024-05-28 11:58:31,84.200000,200.0,NaN,NaN,NaN,NaN,43110,43111,43110.5,229.500000
4,2024-05-28 11:58:34,2024-05-28 11:58:35,82.400000,200.0,NaN,NaN,NaN,NaN,43114,43115,43114.5,258.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
23205,2024-06-17 13:09:21,2024-06-17 13:09:33,84.353846,200.0,1.519434,1.496881,1.757231,NaN,47361,47373,47367.0,2996.666667
23209,2024-06-17 13:12:25,2024-06-17 13:12:44,85.015000,200.0,1.519434,1.496881,1.751957,NaN,47545,47564,47554.5,2978.588235
23234,2024-06-17 13:16:13,2024-06-17 13:16:20,83.875000,200.0,1.519434,1.496881,1.620886,NaN,47773,47780,47776.5,809.272727
23235,2024-06-17 13:16:20,2024-06-17 13:16:26,85.557143,200.0,1.490566,1.469741,1.619950,NaN,47780,47786,47783.0,617.403846


In [16]:
d_test

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_160,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time,GPS_Altitude
124,2024-05-28 15:36:10,2024-05-28 15:36:11,86.600000,160.0,1.369706,1.624879,2.111415,NaN,56170,56171,56170.5,172.000000
125,2024-05-28 15:36:11,2024-05-28 15:36:12,86.350000,160.0,1.369706,1.624879,1.735466,NaN,56171,56172,56171.5,172.000000
126,2024-05-28 15:36:12,2024-05-28 15:36:13,85.850000,160.0,1.369706,1.624879,2.108478,NaN,56172,56173,56172.5,172.000000
127,2024-05-28 15:36:13,2024-05-28 15:36:14,85.150000,160.0,1.369706,1.624879,2.293516,NaN,56173,56174,56173.5,172.000000
128,2024-05-28 15:36:14,2024-05-28 15:36:15,84.550000,160.0,1.369706,1.624879,1.735466,NaN,56174,56175,56174.5,172.428571
...,...,...,...,...,...,...,...,...,...,...,...,...
957,2024-05-28 17:04:18,2024-05-28 17:04:30,84.546154,160.0,1.403169,1.480070,2.293516,NaN,61458,61470,61464.0,NaN
966,2024-05-28 17:07:51,2024-05-28 17:07:57,84.200000,160.0,NaN,NaN,NaN,NaN,61671,61677,61674.0,NaN
967,2024-05-28 17:08:14,2024-05-28 17:08:23,85.050000,160.0,NaN,NaN,NaN,NaN,61694,61703,61698.5,NaN
1019,2024-05-28 17:21:35,2024-05-28 17:21:42,84.762500,160.0,NaN,NaN,NaN,NaN,62495,62502,62498.5,NaN


In [7]:
r_dps[0]

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_160,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time
124,2024-05-28 15:36:10,2024-05-28 15:36:11,86.600000,160.0,1.369706,1.624879,2.111415,NaN,56170,56171,56170.5
125,2024-05-28 15:36:11,2024-05-28 15:36:12,86.350000,160.0,1.369706,1.624879,1.735466,NaN,56171,56172,56171.5
126,2024-05-28 15:36:12,2024-05-28 15:36:13,85.850000,160.0,1.369706,1.624879,2.108478,NaN,56172,56173,56172.5
127,2024-05-28 15:36:13,2024-05-28 15:36:14,85.150000,160.0,1.369706,1.624879,2.293516,NaN,56173,56174,56173.5
128,2024-05-28 15:36:14,2024-05-28 15:36:15,84.550000,160.0,1.369706,1.624879,1.735466,NaN,56174,56175,56174.5
...,...,...,...,...,...,...,...,...,...,...,...
957,2024-05-28 17:04:18,2024-05-28 17:04:30,84.546154,160.0,1.403169,1.480070,2.293516,NaN,61458,61470,61464.0
966,2024-05-28 17:07:51,2024-05-28 17:07:57,84.200000,160.0,NaN,NaN,NaN,NaN,61671,61677,61674.0
967,2024-05-28 17:08:14,2024-05-28 17:08:23,85.050000,160.0,NaN,NaN,NaN,NaN,61694,61703,61698.5
1019,2024-05-28 17:21:35,2024-05-28 17:21:42,84.762500,160.0,NaN,NaN,NaN,NaN,62495,62502,62498.5
